# Aurora — Mathematical Audit Notebook

Аудит критичних та складних математичних компонентів Aurora: TCA-тотожності, знакові конвенції, хвости ризику (EVT), статистична валідація (bootstrap CI, DM-тест), walk-forward стабільність. Артефакти пишуться до `/content/drive/MyDrive/reports/`.


## Setup (Colab) — інструкції
- Натисни ▶ у наступній комірці, щоб встановити залежності (якщо потрібно).
- Дочекайся зеленої галочки. Далі переходь до Drive/Paths.


In [ ]:
IN_COLAB = 'google.colab' in __import__('sys').modules
if IN_COLAB:
    %pip -q install pandas numpy matplotlib seaborn tomli || true
import numpy as np, pandas as pd, json, math, os, gzip, io, statistics, time
import matplotlib.pyplot as plt; import seaborn as sns; sns.set(style='whitegrid')
from pathlib import Path


## Paths (Google Drive) — інструкції
- В Colab поклади дані у `/content/drive/MyDrive/datasets/`, конфіги — у `/content/drive/MyDrive/configs/`.
- Натисни ▶ — підключимо Drive та зафіксуємо ROOT.


In [ ]:
ROOT = Path('/content/drive/MyDrive') if IN_COLAB else Path.cwd()
if IN_COLAB:
    try:
        from google.colab import drive as _drive
        _drive.mount('/content/drive', force_remount=False)
    except Exception as e:
        print('Drive mount warning:', e)
REPORTS = ROOT/'reports'; REPORTS.mkdir(parents=True, exist_ok=True)
DATASETS = ROOT/'datasets'; DATASETS.mkdir(parents=True, exist_ok=True)
CONFIGS = ROOT/'configs'
print('Using ROOT =', ROOT)


## Load Config + Logs — інструкції
- Конфіги очікуємо у `/content/drive/MyDrive/configs/` (`default.toml`, `schema.json`).
- Логи шукаємо у `/content/drive/MyDrive/datasets/` (`*.jsonl.gz` або `*.jsonl`). Якщо немає — зʼявиться віджет завантаження.
- Натисни ▶.


In [ ]:
def _toml_load_bytes(b):
    try:
        import tomllib; return tomllib.loads(b.decode('utf-8') if isinstance(b,(bytes,bytearray)) else b)
    except Exception:
        import tomli; return tomli.loads(b.decode('utf-8') if isinstance(b,(bytes,bytearray)) else b)

def load_ssot(configs_dir: Path):
    cfg = configs_dir/'default.toml'
    if cfg.exists(): return _toml_load_bytes(cfg.read_bytes())
    return {}

def read_jsonl_any(p: Path):
    buf='';
    try:
        if str(p).endswith('.gz'):
            import gzip
            with gzip.open(p,'rt',encoding='utf-8') as f: buf=f.read()
        else:
            buf=p.read_text(encoding='utf-8')
    except Exception:
        return []
    out=[]
    for line in buf.splitlines():
        try: out.append(json.loads(line))
        except Exception: pass
    return out

CFG = load_ssot(CONFIGS)
found = sorted(DATASETS.rglob('*.jsonl.gz')) + sorted(DATASETS.rglob('*.jsonl'))
LOG_FILES = found[:200]
if IN_COLAB and not LOG_FILES:
    try:
        from google.colab import files
        up = files.upload(); LOG_FILES=[Path(k) for k in up.keys()]
    except Exception: pass
rows=[]
for p in LOG_FILES: rows.extend(read_jsonl_any(p))
df = pd.json_normalize(rows) if rows else pd.DataFrame()
print('Configs loaded:', bool(CFG), '| Logs files:', len(LOG_FILES), '| Rows:', len(df))


## TCA Identity & Sign Conventions — інструкції
- Натисни ▶ — перевіримо: IS = raw + fees + slippages + adverse + latency + impact + rebate.
- Перевіримо знаки компонентів (≤0 або ≥0, як належить). Отримаємо агреговану статистику порушень.


In [ ]:
checks = {}
if df.empty:
    print('No data to audit.')
else:
    cols = ['is_bps','raw_edge_bps','fees_bps','slippage_in_bps','slippage_out_bps','adverse_bps','latency_bps','impact_bps','rebate_bps']
    present = [c for c in cols if c in df.columns]
    if 'is_bps' not in df.columns and 'IS_bps' in df.columns: df['is_bps']=pd.to_numeric(df['IS_bps'],errors='coerce')
    for c in cols:
        if c not in df.columns: df[c]=np.nan
    X = df.copy()
    for c in cols: X[c]=pd.to_numeric(X[c],errors='coerce')
    comp_sum = X['raw_edge_bps']+X['fees_bps']+X['slippage_in_bps']+X['slippage_out_bps']+X['adverse_bps']+X['latency_bps']+X['impact_bps']+X['rebate_bps']
    diff = (X['is_bps'] - comp_sum).abs()
    tol = 1e-6
    id_ok = (diff<=tol).sum(); id_bad = (diff>tol).sum()
    # Sign gates
    gates = {
      'fees_bps<=0': (X['fees_bps']<=1e-12).mean(),
      'slip_in<=0': (X['slippage_in_bps']<=1e-12).mean(),
      'slip_out<=0': (X['slippage_out_bps']<=1e-12).mean(),
      'adverse<=0': (X['adverse_bps']<=1e-12).mean(),
      'latency<=0': (X['latency_bps']<=1e-12).mean(),
      'impact<=0': (X['impact_bps']<=1e-12).mean(),
      'rebate>=0': (X['rebate_bps']>=-1e-12).mean(),
    }
    checks = {'identity_ok': int(id_ok), 'identity_bad': int(id_bad), 'gates_share': gates}
    print(json.dumps(checks, indent=2))
    (REPORTS/'audit_tca_checks.json').write_text(json.dumps(checks, indent=2), encoding='utf-8')


## Bootstrap CI (IS, Sharpe) — інструкції
- Натисни ▶ — отримаєш довірчі інтервали для середнього IS та Sharpe (простий bootstrap).
- Якщо даних мало, інтервали будуть широкі.


In [ ]:
def bootstrap_ci(series: np.ndarray, func, B=1000, alpha=0.05, random_state=123):
    rng = np.random.default_rng(random_state)
    series = series[~np.isnan(series)]
    if len(series)==0: return (np.nan,np.nan,np.nan)
    stats = []
    n=len(series)
    for _ in range(B):
        idx = rng.integers(0,n,size=n)
        stats.append(func(series[idx]))
    stats = np.array(stats)
    lo,hi = np.quantile(stats,[alpha/2,1-alpha/2])
    return float(np.mean(stats)), float(lo), float(hi)

def sharpe(series: np.ndarray):
    x = series[~np.isnan(series)]
    if len(x)<2: return np.nan
    return float(np.mean(x)/(np.std(x,ddof=1)+1e-12))

if df.empty or ('is_bps' not in df.columns and 'IS_bps' not in df.columns):
    print('No IS data for bootstrap.')
else:
    s = pd.to_numeric(df['is_bps'] if 'is_bps' in df.columns else df['IS_bps'], errors='coerce').values
    m_mu, m_lo, m_hi = bootstrap_ci(s, lambda z: np.mean(z))
    sh, sh_lo, sh_hi = bootstrap_ci(s, sharpe)
    res = {'IS_mean': {'est':m_mu,'lo':m_lo,'hi':m_hi}, 'Sharpe': {'est':sh,'lo':sh_lo,'hi':sh_hi}}
    print(json.dumps(res, indent=2))
    (REPORTS/'audit_bootstrap.json').write_text(json.dumps(res, indent=2), encoding='utf-8')


## EVT Tail Audit (POT) — інструкції
- Натисни ▶ — оцінимо хвіст за методом Peaks-Over-Threshold (спрощено).
- Отримаємо VaR/CVaR емпірично, mean-excess діаграму, прості оцінки.


In [ ]:
def empirical_var_cvar(x, alpha=0.95):
    x = np.sort(x)
    n = len(x)
    if n==0: return (np.nan,np.nan)
    k = max(1,int((1-alpha)*n))
    tail = x[:k]  # worst side assuming IS positive is good; if working with losses use sign accordingly
    var = x[k-1] if k>0 else x[0]
    cvar = float(np.mean(tail)) if k>0 else var
    return (var,cvar)

if df.empty or ('is_bps' not in df.columns and 'IS_bps' not in df.columns):
    print('No IS data for EVT.')
else:
    s = pd.to_numeric(df['is_bps'] if 'is_bps' in df.columns else df['IS_bps'], errors='coerce').dropna().values
    # Для downside tail беремо мінімальні значення IS (гірша сторона)
    s_sorted = np.sort(s)
    alpha=0.95
    k = max(1,int((1-alpha)*len(s_sorted)))
    tail = s_sorted[:k]
    if len(tail)==0: print('Tail too small.'); tail=np.array([])
    # Mean-excess plot
    if len(s_sorted)>10:
        qs = np.linspace(0.8,0.99,20)
        us = [np.quantile(s_sorted,q) for q in qs]
        me = []
        for u in us:
            exc = s_sorted[s_sorted<=u]  # worst side
            me.append(float(np.mean(u-exc)) if len(exc)>0 else np.nan)
        plt.figure(figsize=(5,3)); plt.plot(us, me, marker='o'); plt.title('Mean Excess (downside)'); plt.xlabel('u'); plt.ylabel('E[u - X | X≤u]'); plt.tight_layout(); plt.savefig(REPORTS/'audit_mean_excess.png', dpi=150); plt.close()
    var95, cvar95 = empirical_var_cvar(s_sorted, alpha=0.95)
    out = {'VaR95_emp': var95, 'CVaR95_emp': cvar95}
    print(json.dumps(out, indent=2))
    (REPORTS/'audit_evt.json').write_text(json.dumps(out, indent=2), encoding='utf-8')


## VaR Backtests (Kupiec/Christoffersen) — інструкції
- Натисни ▶ — перевіримо частоту пробоїв та незалежність.
- Потрібна послідовність прибутків/метрики (використаємо IS).


In [ ]:
def kupiec_pof(x, alpha=0.95):
    # exceedance if x <= VaR_alpha (downside)
    n=len(x); k = max(1,int((1-alpha)*n)); var = np.sort(x)[k-1]
    I = (x<=var).astype(int); N=n; xk=I.sum(); p=1-alpha
    # LR_pof = -2 ln( (1-p)^(N-k) p^k / ((1-xk/N)^(N-k) (xk/N)^k) )
    if xk==0 or xk==N: return {'LR_pof':np.nan,'exceed_rate':xk/N}
    import math
    num = ((1-p)**(N-xk))*(p**xk)
    den = ((1-xk/N)**(N-xk))*((xk/N)**xk)
    lr = -2*math.log(num/den)
    return {'LR_pof': float(lr), 'exceed_rate': float(xk/N)}

def christoffersen_ind(x, alpha=0.95):
    n=len(x); k = max(1,int((1-alpha)*n)); var = np.sort(x)[k-1]
    I = (x<=var).astype(int)
    # transition counts
    n00=n01=n10=n11=0
    for i in range(1,n):
        a,b = I[i-1], I[i]
        if a==0 and b==0: n00+=1
        elif a==0 and b==1: n01+=1
        elif a==1 and b==0: n10+=1
        else: n11+=1
    import math
    pi0 = n01/max(1,(n00+n01)); pi1 = n11/max(1,(n10+n11)); pi = (n01+n11)/max(1,(n00+n01+n10+n11))
    def L(a,b,p):
        return ((1-p)**a)*(p**b) if a>=0 and b>=0 else 1.0
    L0 = L(n00+n10, n01+n11, pi)
    L1 = L(n00, n01, pi0)*L(n10, n11, pi1)
    if L0<=0 or L1<=0: return {'LR_ind':np.nan,'n00':n00,'n01':n01,'n10':n10,'n11':n11}
    LR_ind = -2*math.log(L0/L1)
    return {'LR_ind': float(LR_ind), 'n00':n00,'n01':n01,'n10':n10,'n11':n11}

if df.empty or ('is_bps' not in df.columns and 'IS_bps' not in df.columns):
    print('No IS data for VaR tests.')
else:
    s = pd.to_numeric(df['is_bps'] if 'is_bps' in df.columns else df['IS_bps'], errors='coerce').dropna().values
    out = {'kupiec': kupiec_pof(s,0.95), 'christoffersen': christoffersen_ind(s,0.95)}
    print(json.dumps(out, indent=2))
    (REPORTS/'audit_var_tests.json').write_text(json.dumps(out, indent=2), encoding='utf-8')


## Walk-Forward Stability — інструкції
- Натисни ▶ — розібʼємо дані на k-вікон по часу (за `ts_ms`/індексом), порахуємо EΠ та CVaR по кожному вікну.
- Це показує стабільність/крихкість результатів у часі.


In [ ]:
def cvar_downside(x, alpha=0.95):
    x=np.sort(x); n=len(x); k=max(1,int((1-alpha)*n)); return float(np.mean(x[:k])) if k>0 else (x[0] if n>0 else np.nan)

def walk_forward(df, k=5):
    if df.empty: return []
    # time-key
    if 'ts_ms' in df.columns: t = pd.to_numeric(df['ts_ms'], errors='coerce').fillna(method='ffill').fillna(0)
    elif 'ts' in df.columns: t = pd.to_numeric(df['ts'], errors='coerce').fillna(method='ffill').fillna(0)
    else: t = pd.Series(range(len(df)))
    order = np.argsort(t.values)
    idx_chunks = np.array_split(order, k)
    series = pd.to_numeric(df['is_bps'] if 'is_bps' in df.columns else df.get('IS_bps', np.nan), errors='coerce').values
    out=[]
    for i,chunk in enumerate(idx_chunks):
        s = series[chunk]
        s = s[~np.isnan(s)]
        if len(s)==0: out.append({'win':i,'mean_IS':np.nan,'CVaR95':np.nan}); continue
        out.append({'win':i,'mean_IS': float(np.mean(s)), 'CVaR95': float(cvar_downside(s,0.95))})
    return out

wf = walk_forward(df, k=5)
print(json.dumps(wf, indent=2))
(REPORTS/'audit_walk_forward.json').write_text(json.dumps(wf, indent=2), encoding='utf-8')


## Audit Summary — інструкції
- Натисни ▶ — зберемо коротке резюме /content/drive/MyDrive/reports/audit_summary.md.


In [ ]:
lines=[
  '# Aurora Audit Summary',
  '',
  'Artifacts:',
  '- audit_tca_checks.json — TCA тотожність та знакові конвенції',
  '- audit_bootstrap.json — Bootstrap CI (IS, Sharpe)',
  '- audit_evt.json — EVT (емпіричні VaR/CVaR, mean-excess.png)',
  '- audit_var_tests.json — Kupiec/Christoffersen тести',
  '- audit_walk_forward.json — walk-forward стабільність',
]
(REPORTS/'audit_summary.md').write_text('
'.join(lines), encoding='utf-8')
print('Wrote', REPORTS/'audit_summary.md')
